In [1]:
import numpy as np
import argparse
import cv2

from tqdm import tqdm
import sys
sys.path.append("/Users/rcarlson/Documents/GitHub/lasagna3/snakes")
import mahotas
from ops.imports_ipython import *
from scipy.spatial.distance import cdist


import skimage
import tensorflow as tf

/home/rcarlson/OpticalPooledScreens/ops/annotate.py:20: UserWarning: visitor font not found at /home/rcarlson/OpticalPooledScreens/ops/visitor1.ttf
  warnings.warn('visitor font not found at {0}'.format(VISITOR_PATH))


## Run pretrained network

In [2]:
from keras.applications import imagenet_utils
from keras.applications import Xception #

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.models import Model

# Source model
model = Xception(weights = 'imagenet')

# remove last layer
bottleneck_model = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print(bottleneck_model.summary())


import gc

def preprocess_img(imgname):
        print(imgname)
        imgs = (read(imgname))
        print(imgs.shape)
        mean = imgs.mean(axis = (1,2))
        sd = imgs.std(axis = (1,2))
        mins = imgs.min(axis = (1,2))
        maxs = imgs.max(axis = (1,2))
        
        mean = np.mean(mean)
        sd = np.mean(sd)
        
        imgs = (imgs - mean)/sd

        imlist = []
        for i in range(imgs.shape[0]):
            if i % 500 == 0:
                print(i)
            img = imgs[i] - mins[i]
            img = img * 255/maxs[i]
            img = np.expand_dims(img, axis = -1)
            img = np.repeat(img, 3, axis = -1)
            img = keras.applications.xception.preprocess_input(img)
            img = np.expand_dims(img,axis = 0)
            imlist.append(img)
        
        print('preprocessing done')

        return imlist
    
def predict_features(imgs):
    imgs = tf.data.Dataset.from_tensor_slices(imgs)
    print('tf dataset created')
    features = bottleneck_model.predict(imgs, use_multiprocessing = True, batch_size = 100, verbose = 1, workers = 2)
    return features


from tqdm.contrib.concurrent import process_map
import keras
def batch_preprocess_predict(file, start, end, plate, well):
    imgs = preprocess_img(file)
    print(len(imgs))

    feats = predict_features(imgs)
    (pd.DataFrame(feats)
        .to_pickle('/home/rcarlson/mountb/m40-m42/process/xception/feats_'+plate+'_'+well+'_'+str(start)+'_'+str(end)+'.ch'+str(channel)+'.pkl'))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [ ]:
import gc


files =  natsorted(glob('/home/rcarlson/mountb/m40-m42/process/single_cells/*.tif'))
for file in files:
    channel = file.split('ch')[1][0]
    print('ch: ', channel)
    plate = file.split('/')[-1].split('_')[0]
    well = file.split('/')[-1].split('_')[3]
    start = file.split('/')[-1].split('_')[1]
    end = file.split('/')[-1].split('_')[2]

    print(start, end, plate, well)
    batch_preprocess_predict(file, start=start,end=end, plate=plate, well=well) ## 
    gc.collect()